In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

fig_width = 960
fig_write = True

In [2]:
coureurs = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Coureurs")
motor = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Motor")
chassis = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Chassis")
teams = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Teams")

races = list(coureurs.columns[4:4+2])

In [3]:
def calc_score_coureur(i, race):
    score = coureurs.loc[coureurs.index == i][race] * 0.5
    return score.values[0]

def calc_score_chassis(i, race):
    score = coureurs.loc[coureurs['chassis'] == i][race]
    return score.sum() * 0.2
    
def calc_score_motor(i, race):
    score = coureurs.loc[coureurs['motor'] == i][race]
    return score.sum() / (0.5*score.count()) * 0.3

def set_bg_color(fig):
    fig.update_layout(plot_bgcolor='rgba(224, 224, 224, 1)')
    fig.update_layout(paper_bgcolor='rgba(224, 224, 224, 1)')
    fig.update_yaxes(showgrid=True)

In [4]:
df = pd.DataFrame(columns=['team', 'race', 'cat', 'score'])

for race in races:
    score_coureur1 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur1'] ])
    score_coureur2 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur2'] ])
    score_chassis  = pd.Series([ calc_score_chassis(i, race) for i in teams['chassis' ] ])
    score_motor    = pd.Series([ calc_score_motor  (i, race) for i in teams['motor'   ] ])
    
    df_coureur1 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur1', 'score' : score_coureur1})
    df_coureur2 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur2', 'score' : score_coureur2})
    df_chassis = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'chassis', 'score' : score_chassis})
    df_motor = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'motor', 'score' : score_motor})
    
    df = pd.concat([df, df_coureur1, df_coureur2, df_chassis, df_motor])

In [5]:
df_total_score = df.groupby(['team'], as_index=False)['score'].sum()
df_total_score = df_total_score.sort_values(by='score', ascending=False)
# df_total_score = df_total_score.sort_values(by='team', ascending=True)

In [8]:
df_race = df.loc[df['race']=='Bahrein']
df_race = df_race.sort_values(by="team")
fig = px.histogram(df_race, x="team", y="score", color="cat",
            title="Resultaten Bahrein",
            width=fig_width, height=400,
            labels={ "score" : "Score", "cat" : "Categorie", "team" : "Teams" },
            template="simple_white")

fig.update_layout(yaxis_title="Punten")

set_bg_color(fig)

series_name = ["Coureur 1", "Coureur 2", "Chassis", "Motor"]
for idx, name in enumerate(series_name):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate=name + "<br>Punten = %{y}<extra></extra>"
#    fig.data[idx].hovertemplate = name

if fig_write:
    fig.write_html("resultaten_bahrein.html", full_html=False, include_plotlyjs='cdn')
fig.show()

In [9]:
df_race = df.loc[df['race']=='Saudi-Arabië']
df_race = df_race.sort_values(by="team")
fig = px.histogram(df_race, x="team", y="score", color="cat",
            title="Resultaten Saudi-Arabië",
            width=fig_width, height=400,
            labels={ "score" : "Score", "cat" : "Categorie", "team" : "Teams" },
            template="simple_white")

fig.update_layout(yaxis_title="Punten")

set_bg_color(fig)

series_name = ["Coureur 1", "Coureur 2", "Chassis", "Motor"]
for idx, name in enumerate(series_name):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate=name + "<br>Punten = %{y}<extra></extra>"
#    fig.data[idx].hovertemplate = name

if fig_write:
    fig.write_html("resultaten_saudi_arabie.html", full_html=False, include_plotlyjs='cdn')
fig.show()

In [10]:
df_score_per_race = df.groupby(['race', 'team'])['score'].sum(numeric_only=True).unstack()#.reset_index()#.rename_axis(None, axis=1)
# df_score_per_race.transpose()
df_score_per_race = df_score_per_race.reindex(races)
df_score_cumsum = df_score_per_race.cumsum()

In [11]:
discrete_colors = px.colors.sample_colorscale('Bluered', np.arange(0.5, 21.5) / 22)

fig = px.line(
    df_score_cumsum, template='simple_white',
    title="Stand",
    labels={ "team" : "Teams" },
    width=fig_width, height=660)#, color_discrete_map=discrete_colors)

fig.update_layout(xaxis_title="Races") 
fig.update_layout(yaxis_title="Puntentotaal")
set_bg_color(fig)

for idx, name in enumerate(fig.data):
    fig.data[idx].hovertemplate = fig.data[idx].name + "<br>Race = %{x}<br>Punten = %{y}<extra></extra>"

if fig_write:
    fig.write_html("totale_punten_lijn.html", full_html=False, include_plotlyjs='cdn')
fig.show()

In [12]:
team_names = df_total_score['team']

fig = px.histogram(df, x="team", y="score", color="race",
            title="Stand",
            width=fig_width, height=660,
            labels={ "race" : "Races" },
            template="simple_white"
            )

fig.update_layout(xaxis_title="Teams") 
fig.update_layout(yaxis_title="Puntentotaal")

fig.update_xaxes(categoryorder='array', categoryarray=team_names)

set_bg_color(fig)

for idx, name in enumerate(races):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate = name + "<br>Punten = %{y}<extra></extra>"       
                 
if fig_write:
    fig.write_html("totale_punten_staaf.html", full_html=False, include_plotlyjs='cdn')
fig.show()

In [13]:
team_names = df_total_score['team']

fig = px.histogram(df, x="team", y="score", color="race",
            title="Stand",
            width=fig_width, height=660,
            labels={ "race" : "Races" },
            template="simple_white"
            )

fig.update_layout(xaxis_title="Teams") 
fig.update_layout(yaxis_title="Puntentotaal")

fig.update_xaxes(categoryorder='array', categoryarray=team_names)

set_bg_color(fig)

for idx, name in enumerate(races):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate = name + "<br>Punten = %{y}<extra></extra>"       
                 
if fig_write:
    fig.write_html("totale_punten_staaf.html", full_html=False, include_plotlyjs='cdn')
fig.show()